In [1]:
!pip install git+https://github.com/allenai/ir_datasets.git
!python -m spacy download en_core_web_sm
!pip install spacy

  Cloning https://github.com/allenai/ir_datasets.git to /tmp/pip-req-build-hnunkcgl
  Running command git clone --filter=blob:none -q https://github.com/allenai/ir_datasets.git /tmp/pip-req-build-hnunkcgl
  Resolved https://github.com/allenai/ir_datasets.git to commit 93bb9406914d2bd4a59c5dc3ff8627611d4fe600
  Preparing metadata (setup.py) ... done
2022-01-11 14:11:58.381341: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-11 14:11:58.381388: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
     |████████████████████████████████| 13.9 MB 4.5 MB/s            


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [2]:
import ir_datasets
dataset = ir_datasets.load('argsme/2020-04-01')
import os
# From https://github.com/grill-lab/trec-cast-tools/blob/master/src/main/python/passage_chunker.py
import re
import spacy
import numpy as np

nlp = spacy.load("en_core_web_sm", exclude=["parser", "tagger", "ner", "attribute_ruler", "lemmatizer", "tok2vec"]) #->senter
nlp.enable_pipe("senter")

print(nlp.pipe_names)


class SpacyPassageChunker:
    def __init__(self):
        self.document = None
        self.sentences = None

    
    def sentence_tokenization(self, document):
        self.document = parse_doc(document)
        self.sentences = list(self.document.sents)

    
    def create_passages(self, passage_size = 250):

        passages = []
        content_length = len(self.sentences)
        sentences_word_count = [len([token for token in sentence]) for sentence in self.sentences]
        
        current_idx = 0
        current_passage_word_count = 0
        current_passage = ''
        sub_id = 0
        
        for i in range(content_length):
            if current_passage_word_count >= (passage_size * 0.67):
                passages.append({
                    "body": current_passage,
                    "id": sub_id
                })
                current_passage = ''
                current_passage_word_count = 0
                
                current_idx = i
                sub_id += 1
            
            current_passage += self.sentences[i].text + ' '
            current_passage_word_count += sentences_word_count[i]

        
        current_passage = ' '.join(sentence.text for sentence in self.sentences[current_idx:])
        passages.append({
            "body": current_passage,
            "id": sub_id
        })
        
        return passages
        
        
def parse_doc(document):
    return nlp(sanitize_document(document))

def sanitize_document(doc):
    sanitized = re.compile('<.*?>')
    return re.sub(sanitized, '', doc)[:999999]

def tokens(doc):
    return set([i for i in parse_doc(doc) if not i.is_punct])


def passages(doc):

    ret = [] #Originally "ret = [doc.conclusion]" but since conclusion is Reference Field it shouldnt be saved.
    tmp = SpacyPassageChunker()
    tmp.sentence_tokenization(doc.premises_texts)

    ret += [i['body'] for i in tmp.create_passages(passage_size=300)]

    return [(i[0] +1, i[1]) for i in enumerate(ret)]

import json
from tqdm import tqdm
DIR = '/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_data'

def docs_iter(dataset_name):
    import ir_datasets
    iter = ir_datasets.load(dataset_name).docs_iter()
    for doc in iter:
        yield doc

import json

# dataset_name = 'argsme'
labeled_arguments_ids = np.load('labeled_arguments.npy')
with open(os.path.join(DIR, 'argsme2020-04-01-passages-without-conclusions-and-leakage.jsonl'), 'w') as f:
#     # for doc in dataset.docs_iter():
    for doc in tqdm(dataset.docs_iter(), total=387740):
        if doc.doc_id not in labeled_arguments_ids:
            f.write(json.dumps({'id': doc.doc_id, 'passages': passages(doc)}) + '\n')



2022-01-11 14:13:42.433440: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-11 14:13:42.433492: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


['senter']


100%|██████████████████████████████████████████████████████████████████████████| 387740/387740 [14:16<00:00, 452.62it/s]
